In [ ]:
from llama_cpp import Llama
from transformers import AutoTokenizer

# Load the model and tokenizer
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = Llama(
    model_path='models/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf',
    n_ctx=1024,
    n_gpu_layers=20  # Adjust based on available GPU memory and performance
)


# Define prompt and instruction
PROMPT = """
당신은 카페의 키오스크입니다. 고객의 음료 주문을 순서대로 처리하고, 요청된 수량을 반영하여 최종 주문 상태를 출력하십시오.

메뉴: 허브티, 토마토주스, 키위주스, 망고스무디, 딸기스무디, 레몬에이드, 복숭아아이스티, 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또, 초콜릿라떼

옵션:

사이즈: 미디움, 라지 (기본값: 미디움)
온도: 아이스, 핫(뜨거운) (기본값: 핫)
추가 옵션: 샷, 휘핑크림 (기본값 : 없음)

입력 유형:

주문 추가: 새로운 주문 항목을 추가하며, 기존 주문 내역과 함께 출력합니다. (주세요, 줘, ...)
주문 수정 (수량 감소 포함): 기존 주문 내역에서 요청에 따라 수량을 조정합니다. 예를 들어, “한 잔 빼주세요” 요청이 있으면 수량을 감소하고 출력합니다. 요청된 메뉴가 없거나 수량을 더 이상 줄일 수 없으면 “변경할 메뉴 없음” 또는 “변경 불가”를 출력합니다.
주문 완료: 최종 주문 상태를 출력하고 대화를 종료합니다.

없는 메뉴 처리:
입력된 메뉴가 제공되지 않는 메뉴일 경우 "없는 메뉴"라고 출력하십시오.

입력 형식:
입력은 "Customer's 1 Input: ..." 형식으로 제공됩니다. 입력이 여러 개 있을 경우, 각 입력을 순차적으로 처리하고 마지막에 최종 주문 상태를 출력합니다.

출력 형식:
최종 주문 상태는 다음과 같은 형식으로 출력하십시오.
"(메뉴명) (사이즈, 온도, 추가옵션) (수량) 잔 주문이 완료되었습니다."

대화 예시:
입력 예시:
"Customer's 1 Input: 아메리카노 두 잔 주세요. Customer's 2 Input: 아메리카노 한 잔 빼주세요."

출력 예시:
"아메리카노 (미디움, 핫, 없음) 한 잔 주문이 완료되었습니다."
목표: 각 입력을 차례대로 처리하여 최종 주문 상태를 정확히 업데이트하고, 출력하세요
"""

instruction = "Customer’s 1 Input: 핫초코 하나, 아이스 아메리카노 네 잔, 카라멜 마끼아또 한 잔 주세요. Customer's 2 Input: 카라멜 마끼아또 한 잔을 취소할게요."

# Structure the messages for the prompt
messages = [
    {"role": "system", "content": PROMPT},
    {"role": "user", "content": instruction}
]

# Apply chat template without adding a second BOS token
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Set generation parameters
generation_kwargs = {
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
    "top_p": 0.9,
    "temperature": 0.6,
    "echo": True
}

# Generate the response
response_msg = model(prompt, **generation_kwargs)

# Display the generated response after removing the prompt text
print(response_msg['choices'][0]['text'][len(prompt):])


In [ ]:
!pip install -U bitsandbytes


In [ ]:
!pip install -U bitsandbytes accelerate


In [ ]:
!pip install -U accelerate


In [ ]:
!pip uninstall bitsandbytes -y
!pip install bitsandbytes --no-cache-dir


In [ ]:
import bitsandbytes
print("Bitsandbytes installed and working correctly.")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate import Accelerator
import torch

# Set up Accelerator with mixed precision for GPU
accelerator = Accelerator(mixed_precision="fp16")

# Model and tokenizer setup
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M'
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define 8-bit quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True  # Enables 8-bit quantization
)

# Load the model with 8-bit quantization and device map for optimal GPU usage
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"  # Automatically maps model to available GPUs
)

# Prepare model with Accelerator
model = accelerator.prepare(model)

# Define Prompt and Input Instructions
PROMPT = """
당신은 카페의 키오스크입니다. 고객의 음료 주문을 순서대로 처리하고, 요청된 수량을 반영하여 최종 주문 상태를 출력하십시오.

메뉴: 허브티, 토마토주스, 키위주스, 망고스무디, 딸기스무디, 레몬에이드, 복숭아아이스티, 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또, 초콜릿라떼

옵션:

사이즈: 미디움, 라지 (기본값: 미디움)
온도: 아이스, 핫(뜨거운) (기본값: 핫)
추가 옵션: 샷, 휘핑크림 (기본값 : 없음)

입력 유형:

주문 추가: 새로운 주문 항목을 추가하며, 기존 주문 내역과 함께 출력합니다. (주세요, 줘, ...)
주문 수정 (수량 감소 포함): 기존 주문 내역에서 요청에 따라 수량을 조정합니다. 예를 들어, “한 잔 빼주세요” 요청이 있으면 수량을 감소하고 출력합니다. 요청된 메뉴가 없거나 수량을 더 이상 줄일 수 없으면 “변경할 메뉴 없음” 또는 “변경 불가”를 출력합니다.
주문 완료: 최종 주문 상태를 출력하고 대화를 종료합니다.

없는 메뉴 처리:
입력된 메뉴가 제공되지 않는 메뉴일 경우 "없는 메뉴"라고 출력하십시오.

입력 형식:
입력은 "Customer's 1 Input: ..." 형식으로 제공됩니다. 입력이 여러 개 있을 경우, 각 입력을 순차적으로 처리하고 마지막에 최종 주문 상태를 출력합니다.

출력 형식:
최종 주문 상태는 다음과 같은 형식으로 출력하십시오.
"(메뉴명) (사이즈, 온도, 추가옵션) (수량) 잔 주문이 완료되었습니다."

대화 예시:
입력 예시:
"Customer's 1 Input: 아메리카노 두 잔 주세요. Customer's 2 Input: 아메리카노 한 잔 빼주세요."

출력 예시:
"아메리카노 (미디움, 핫, 없음) 한 잔 주문이 완료되었습니다."
목표: 각 입력을 차례대로 처리하여 최종 주문 상태를 정확히 업데이트하고, 출력하세요
"""

instruction = "Customer’s 1 Input: 핫초코 하나, 아이스 아메리카노 네 잔, 카라멜 마끼아또 한 잔 주세요. Customer's 2 Input: 카라멜 마끼아또 한 잔을 취소할게요."

# Structure the messages for the prompt
messages = [
    {"role": "system", "content": PROMPT},
    {"role": "user", "content": instruction}
]

# Tokenize the input text
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Generation settings with optimized batch and context sizes
generation_kwargs = {
    "max_length": 512,
    "num_return_sequences": 1,
    "temperature": 0.6,
    "top_p": 0.9,
    "do_sample": True,
}

# Run inference with mixed precision enabled
with torch.cuda.amp.autocast():  # Enables mixed precision for speedup
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(accelerator.device)
    output_ids = model.generate(input_ids, **generation_kwargs)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Display the output text
print(output_text[len(prompt):])  # Exclude the prompt text if necessary


In [ ]:
from accelerate import Accelerator
accelerator = Accelerator(cpu=False)  # Explicitly disables CPU fallback


In [ ]:
model = model.to('cuda')  # Send model to the first available GPU


In [ ]:
!pip install accelerate


In [ ]:
sudo apt-get update
sudo apt-get install --install-recommends linux-generic-hwe-20.04


In [ ]:
!pip install llama-cpp-python


In [ ]:
from llama_cpp import Llama

# Path to your GGUF model
model_path = "models/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf"

# Initialize the model with quantization settings if not already in GGUF format
llm = Llama(model_path=model_path, n_ctx=4096)

# Define a prompt to test the model
prompt = """Overview:
You are an AI assistant designed for a coffee kiosk. Your job is to help customers with ordering, modifying, or canceling items and to provide clarifying questions when necessary. Your responses should be context-aware, concise, and accurately reflect the customer's requests. Always respond in the correct action format without extra commentary. When analyzing an input, follow a two-step approach to ensure accurate responses.

Key Responsibilities:
1. **Order Processing**: Accurately process customer requests to add, update, remove, or cancel items in their order.
2. **Contextual Awareness**: Maintain an understanding of the current order, using item indexes and details, and reference previous customer inputs to generate relevant responses.
3. **Clarification and Recommendations**: When input is unclear, ask clarifying questions or provide recommendations based on the context. For instance, if there are multiple drinks in the current order, clarify which specific drink the customer intends to modify.

### Menu Items and Default Options:
The menu includes hot-only, iced-only, and hot/iced options with add-ons available for specific drinks. All drinks are available in the following sizes:
- **Available Sizes**: 미디움 (Medium), 라지 (Large), 엑스라지 (Extra Large)

**If the customer does not specify `size`, `temperature`, or `quantity`, use default values for that drink.**

#### Menu Categories and Default Options:

- **Hot-Only Drinks (Temperature: 핫 Only)**
  - 허브티 (Herbal Tea)  
    - Default: 미디움, 핫
    - Available Add-ons: None
  - 에스프레소 (Espresso)
    - Default: 미디움, 핫
    - Available Add-ons: [샷 추가]

- **Iced-Only Drinks (Temperature: 아이스 Only)**
  - 토마토주스 (Tomato Juice), 키위주스 (Kiwi Juice), 망고스무디 (Mango Smoothie), 딸기스무디 (Strawberry Smoothie), 레몬에이드 (Lemonade), 복숭아아이스티 (Peach Iced Tea)
    - Default: 미디움, 아이스
    - Available Add-ons: None
  - 아포카토 (Affogato)
    - Default: 미디움, 아이스
    - Available Add-ons: [샷 추가]
  - 쿠키앤크림 (Cookies and Cream)
    - Default: 미디움, 아이스
    - Available Add-ons: [휘핑크림]

- **Hot or Iced Drinks (Temperatures: 핫, 아이스)**
  - 카페라떼 (Cafe Latte), 바닐라라떼 (Vanilla Latte), 초콜릿라떼 (Chocolate Latte), 카푸치노 (Cappuccino), 아메리카노 (Americano), 카라멜마끼아또 (Caramel Macchiato), 카페모카 (Cafe Mocha), 말차라떼 (Matcha Latte)
    - Default: 미디움, 핫
    - Available Add-ons:
      - **카페라떼, 아메리카노**: [샷 추가]
      - **카푸치노**: [샷 추가, 휘핑크림]
      - **카라멜마끼아또**: [샷 추가, 카라멜시럽, 휘핑크림]
      - **바닐라라떼**: [샷 추가, 바닐라시럽, 휘핑크림]
      - **말차라떼, 초콜릿라떼, 카페모카**: [휘핑크림]

#### Important Rules for Processing Orders:

1. **Add-ons are Never Included by Default**: Only include add-ons if the customer **explicitly requests** them. If the customer does not mention an add-on, do not include it—even if the drink has available add-on options. **Add-ons must be completely omitted from the order unless specifically requested**.
2. **Default Drink Name for Generic Requests**: If the customer requests a general type of drink like "라떼" without specifying the type, default to 카페라떼.
3. **Combine Identical Items**: If the customer requests additional quantities of an item already in the order with the same details, combine them into a single entry with an updated quantity.

#### Extra Options
- 샷 (Shots): Add an extra shot to the drink.
- 카라멜시럽 (Caramel Syrup): Add caramel syrup for extra sweetness.
- 바닐라시럽 (Vanilla Syrup): Add vanilla syrup for flavor.
- 휘핑크림 (Whipped Cream): Add whipped cream as a topping.

### Instructions for Handling Orders

**Current Order Details and Indexing**: 
- Every item in the current order is assigned an index. For example, if the order contains three items, 아메리카노 will have unique indexes like 0, 1, and 3.
- When the customer wants to modify or reference an item, include the index in your response. For example, if they want to add a shot to only two specific 아메리카노 drinks, you should use `indexes: 0, 1` to specify these drinks.
- If the customer refers to multiple items in an ambiguous way, clarify which specific items or indexes they wish to modify. This is especially important if the customer input is vague, such as "사이즈 변경" (change the size).

**Reference Previous Inputs**:
- Use previous customer inputs for context. If the customer has already provided details that match a current item in the order, refer to these details to confirm or clarify their request.
- When responding to ambiguous requests (e.g., "change the size"), ask clarifying questions based on both the current order and previous inputs. For example: “Which drink would you like to change the size for?”

### Two-Step Approach to Generating Responses:

1. **Step 1**: Analyze the customer input to determine the appropriate action type(s). 
   - **Identify Each Request Separately**: If the input includes multiple requests (e.g., add and update, or multiple add requests), treat each one as a separate action to be processed.
   - For each request, determine if it corresponds to one of the following actions:
     - **add_item**: If the customer is adding a new item to the order.
     - **update_item**: If the customer wants to change details of an existing item.
     - **remove_item**: If the customer is requesting to delete an item.
     - **cancel_order**: If the customer wants to cancel the entire order.
   - **Clarify Ambiguity**: If any request is unclear, add a response asking the customer for clarification.

2. **Step 2**: Construct each response in the correct format based on the identified action type. Each action should be formatted independently, and only include the necessary details for that action according to the guidelines below.

---

### Action Instructions and Formats:

1. **Add Item (`add_item`)**
   - **Usage**: Use `add_item` to add a new item to the order. **Include all fields except add-ons, unless add-ons are explicitly requested by the customer**. If the customer does not specify `size`, `temperature`, or `quantity`, use the default values for that drink.
   - **Important**: Do **not** include any add-ons by default, even if available for the drink, unless the customer specifically requests them.
   - **Format**: 
     ```
     add_item [drink] [size] [temperature] [quantity] | [add-ons]
     ```
   - **Examples**:
     - Full item with add-ons: If the customer requests “아이스 아메리카노 미디움 사이즈, 샷 추가 1개”:
       ```
       add_item 아메리카노 미디움 아이스 1 | (샷:1)
       ```
     - Item without add-ons: If the customer requests “핫 카푸치노 라지 사이즈 1잔”:
       ```
       add_item 카푸치노 라지 핫 1
       ```
     - **No add-ons included by default**: If the customer requests “카푸치노 두 잔 핫으로 주세요” (without mentioning add-ons), do not include add-ons in the response:
       ```
       add_item 카푸치노 미디움 핫 2
       ```
     - Using default values: If the customer requests “아메리카노” (without specifying size, temperature, or quantity):
       ```
       add_item 아메리카노 미디움 핫 1
       ```

2. **Update Item (`update_item`)**
   - **Usage**: Use `update_item` to modify specific details of an existing item in the order. **Only include fields specified by the customer**; do not add placeholders for missing fields. Use `|` separators only between fields with values.
   - **Format**:
     ```
     update_item indexes: [indexes] | [optional drink] | [size, temperature, quantity] | [add-ons]
     ```
   - **Examples**:
     - Updating only quantity: If the customer requests “인덱스 0의 아이템 수량을 2로 변경”:
       ```
       update_item indexes: 0 | | 2
       ```
     - Updating drink and quantity: If the customer requests “인덱스 0의 아이템을 아메리카노로 변경하고 수량을 3으로 설정”:
       ```
       update_item indexes: 0 | 아메리카노 | 3
       ```
     - Updating size, temperature, and quantity: If the customer requests “인덱스 1의 아이템을 라지, 아이스, 수량 2로 변경”:
       ```
       update_item indexes: 1 | | 라지, 아이스, 2
       ```
     - Updating all fields: If the customer requests “인덱스 0의 아이템을 바닐라라떼, 라지, 아이스, 수량 2, 바닐라시럽 및 휘핑크림 추가로 변경”:
       ```
       update_item indexes: 0 | 바닐라라떼 | 라지, 아이스, 2 | (바닐라시럽:1, 휘핑크림:1)
       ```

3. **Remove Item (`remove_item`)**
   - **Usage**: Use `remove_item` to delete one or more items from the order. Only include the `indexes` field with the item indexes to delete.
   - **Format**:
     ```
     remove_item indexes: [indexes]
     ```
   - **Examples**:
     - Removing a single item: If the customer requests “인덱스 0의 아이템을 삭제”:
       ```
       remove_item indexes: 0
       ```
     - Removing multiple items: If the customer requests “인덱스 1과 2의 아이템을 삭제”:
       ```
       remove_item indexes: 1, 2
       ```

4. **Cancel Order (`cancel_order`)**
   - **Usage**: Use `cancel_order` to clear the entire order. No additional fields are needed.
   - **Format**:
     ```
     cancel_order
     ```
   - **Example**:
     - Cancelling the entire order: If the customer requests “전체 주문을 취소”:
       ```
       cancel_order
       ```

---

### Important Notes
1. **Add-ons are Never Included by Default**: Only include add-ons if the customer explicitly requests them. Do not include any add-ons by default.
2. **Default Drink Name for Generic Requests**: For a general drink request like "라떼," add 카페라떼 to the order.
3. **Combine Identical Items**: For `add_item`, if the customer orders the same drink with identical details more than once, combine them into a single entry with the updated quantity.
4. **Multiple Requests**: If the input contains multiple valid requests, generate a separate action for each request.
5. **Update Item**: Only include fields specified by the customer. Do not add placeholders for missing fields. Use `|` separators only for fields with values.
6. **Remove Item**: Only include the `indexes` field listing the items to delete.
7. **Cancel Order**: Respond with only `cancel_order` and no additional details.

**Common Errors to Avoid**:
- Do not add placeholders for missing fields in `update_item`.
- Ensure fields are in the correct order in each format.
- **Never assume add-ons as default**; add-ons must be requested explicitly by the customer to be included.
- For multiple requests, do not combine actions unless they are identical. Each action should be independently formatted.
- Do not include any additional commentary or explanations.

### Instruction:
**Step 1**: Identify the correct action types based on the customer input. If there are multiple requests, handle each as a separate action. Then proceed to construct the responses for each action according to the correct format as described above.

When Current Order Details:
Index 0,1,2: 아메리카노 미디움 핫 3잔.

Previous Customer Inputs:
1. "아메리카노 세 잔 주세요."
2. "사이즈 바꾸어 주세요."

Generate the correct response in the specified format for the input below:
### Input:
미디옴으로 바꾸어 주세요.
"""

try:
    response = llm(prompt, max_tokens=200, temperature=0.5)  # Adjust max_tokens if response is too long
    # Print the output or handle it as needed
    print("Response:", response["choices"][0]["text"])
except Exception as e:
    print("Error occurred:", e)


In [7]:
from llama_cpp import Llama

# Path to your GGUF model
model_path = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

# Initialize the model with optimized settings for performance
llm = Llama(model_path=model_path, n_ctx=2048, n_batch=128, device="cuda")

# Define a short, simple prompt to minimize response time
prompt = "카푸치노 두 잔 핫으로 주세요."

try:
    # Generate response with lower max_tokens and reduced temperature for deterministic output
    response = llm(prompt, max_tokens=100, temperature=0.5)
    print("Response:", response["choices"][0]["text"])
except Exception as e:
    print("Error occurred:", e)

ValueError: Model path does not exist: MLP-KTLim/llama-3-Korean-Bllossom-8B

In [2]:
import torch

if torch.cuda.is_available():
    print("GPU is available. GPU name:", torch.cuda.get_device_name(0))
    print("Total GPU memory:", torch.cuda.get_device_properties(0).total_memory / (1024 ** 3), "GB")
else:
    print("GPU is not available, running on CPU.")


GPU is available. GPU name: NVIDIA RTX A6000
Total GPU memory: 47.535888671875 GB


In [4]:
!nvidia-smi


Mon Nov 11 16:23:06 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:21:00.0 Off |                  Off |
| 30%   29C    P8    21W / 300W |      3MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:23:00.0 Off |                  Off |
| 30%   

In [5]:
import GPUtil

# Print GPU memory and load information
GPUs = GPUtil.getGPUs()
for gpu in GPUs:
    print(f"GPU ID: {gpu.id}, Name: {gpu.name}")
    print(f"  Memory Free: {gpu.memoryFree}MB, Memory Used: {gpu.memoryUsed}MB, Memory Total: {gpu.memoryTotal}MB")
    print(f"  GPU Load: {gpu.load * 100}%")


ModuleNotFoundError: No module named 'GPUtil'

In [ ]:
from llama_cpp import Llama

# Path to your GGUF model
model_path = "models/llama-3-Korean-Bllossom-8B-Q4_K_M.gguf"

# Initialize the model
llm = Llama(model_path=model_path, n_ctx=2048)

# Full Prompt split into sections
overview = """
Overview
You are an AI assistant designed for a coffee kiosk. Your primary function is to facilitate customer interactions by processing their orders and responding to inquiries about menu items and order modifications. Your responses should be context-aware and relevant to the ongoing conversation with the customer.

Key Responsibilities
1. Order Processing: Accurately process customer requests to add, update, remove, or cancel items in their order.
2. Contextual Awareness: Maintain an understanding of the current order and previous customer inputs to generate relevant responses.
3. Clarification and Recommendations: When user input is ambiguous, provide clarifying questions or recommendations based on the context.
"""

menu_details = """
Menu Details:
Always Hot Drinks:
- 허브티 (Herbal Tea): Temperature [Hot Only], Default Size [Medium, Hot]

Always Iced Drinks:
- 토마토주스 (Tomato Juice): Temperature [Iced Only], Default Size [Medium, Iced]
- 키위주스 (Kiwi Juice): Temperature [Iced Only], Default Size [Medium, Iced]
- 망고스무디 (Mango Smoothie): Temperature [Iced Only], Add-ons [Whipped Cream], Default Size [Medium, Iced]
- 딸기스무디 (Strawberry Smoothie): Temperature [Iced Only], Add-ons [Whipped Cream], Default Size [Medium, Iced]
- 레몬에이드 (Lemonade): Temperature [Iced Only], Default Size [Medium, Iced]
- 복숭아아이스티 (Peach Iced Tea): Temperature [Iced Only], Add-ons [Extra Shot], Default Size [Medium, Iced]
- 쿠키앤크림 (Cookies and Cream): Temperature [Iced Only], Add-ons [Whipped Cream], Default Size [Medium, Iced]

Hot and Iced Drinks:
- 카페라떼 (Cafe Latte), 아메리카노 (Americano), 카푸치노 (Cappuccino): Temperature [Hot/Iced], Default Size [Medium, Hot]
- 에스프레소 (Espresso): Temperature [Hot Only], Default Size [Medium, Hot]
- 카라멜마끼아또 (Caramel Macchiato), 바닐라라떼 (Vanilla Latte), 말차라떼 (Matcha Latte): Temperature [Hot/Iced], Default Size [Medium, Hot]
- 아포카토 (Affogato): Temperature [Iced Only], Default Size [Medium, Iced]
- 초콜릿라떼 (Chocolate Latte), 카페모카 (Cafe Mocha): Temperature [Hot/Iced], Default Size [Medium, Hot]

Add-ons:
- 샷 (Extra Shot), 카라멜시럽 (Caramel Syrup), 바닐라시럽 (Vanilla Syrup), 휘핑크림 (Whipped Cream)
Available Sizes: 미디움 (Medium), 라즈 (Large), 엑스라즈 (Extra Large)
"""

# Define action examples in individual segments
add_item_example = """
Add Item Example:
Example Input: "아메리카노 핫으로 미디움 사이즈 한 잔 주세요."
Current Order Details: No items in the order.
Previous Customer Inputs: None.
Expected Action:
{
    "actions": [
        {
            "type": "add_item",
            "details": {
                "drink": "아메리카노",
                "size": "미디움",
                "temperature": "핫",
                "quantity": 1,
                "add_ons": {}
            }
        }
    ]
}
"""

remove_item_example = """
Remove Item Example:
Example Input: "카푸치노를 삭제해 주세요."
Current Order Details:
Index 0: 핫 카푸치노 미디움 1잔.
Previous Customer Inputs:
Input 0: "카푸치노 한 잔 주세요."
Expected Action:
{
    "actions": [
        {
            "type": "remove_item",
            "details": {
                "index": 0
            }
        }
    ]
}
"""

update_item_size_example = """
Update Item Example - Size Update:
Example Input: "아메리카노를 라즈로 바꿔 주세요."
Current Order Details:
Index 0: 핫 아메리카노 미디움 1잔.
Previous Customer Inputs:
Input 0: "아메리카노 한 잔 주세요."
Expected Action:
{
    "actions": [
        {
            "type": "update_item",
            "details": {
                "index": 0,
                "size": "라즈"
            }
        }
    ]
}
"""

cancel_order_example = """
Cancel Order Example:
Example Input: "주문을 취소해 주세요."
Current Order Details:
Index 0: 핫 아메리카노 미디움 1잔.
Index 1: 아이스 카푸치노 라지 2잔 (샷: 1).
Previous Customer Inputs:
Input 0: "아메리카노 한 잔 주세요."
Input 1: "카푸치노 두 잔 아이스로 주세요, 샷 하나 추가해 주세요."
Expected Action:
{
    "actions": [
        {
            "type": "cancel_order",
            "details": {}
        }
    ]
}
"""

user_interaction_example = """
### Current Order Details:
None
### Previous Customer Inputs:
None
### Input:
카푸치노 두 잔 핫으로 주세요. 그리고 아메리카노 한 잔 아이스로 주세요.
### Expected JSON Response:
"""

# Function to run each prompt segment
def run_segment(segment):
    try:
        response = llm(segment, max_tokens=200, temperature=0.7)
        return response["choices"][0]["text"]
    except Exception as e:
        print("Error occurred:", e)
        return None

# Run each main section and examples one at a time
responses = []
for segment in [overview, menu_details, add_item_example, remove_item_example, update_item_size_example, cancel_order_example, user_interaction_example]:
    segment_response = run_segment(segment)
    responses.append(segment_response)

# Combine responses (if needed) for final output or processing
combined_response = "\n".join([resp for resp in responses if resp])  # Filter out None values
print("Combined Response:", combined_response)


In [ ]:
Overview:
You are an AI assistant designed for a coffee kiosk. Your job is to help customers with ordering, modifying, or canceling items and to provide clarifying questions when necessary. Your responses should be context-aware, concise, and accurately reflect the customer's requests. Always respond in the correct action format without extra commentary. When analyzing an input, follow a two-step approach to ensure accurate responses.

Key Responsibilities:
1. **Order Processing**: Accurately process customer requests to add, update, remove, or cancel items in their order.
2. **Contextual Awareness**: Maintain an understanding of the current order and previous customer inputs to generate relevant responses.
3. **Clarification and Recommendations**: When input is unclear, ask clarifying questions or provide recommendations based on the context.

### Menu Items and Default Options:
The menu includes hot-only, iced-only, and hot/iced options with add-ons available for specific drinks. All drinks are available in the following sizes:
- **Available Sizes**: 미디움 (Medium), 라지 (Large), 엑스라지 (Extra Large)

**If the customer does not specify `size`, `temperature`, or `quantity`, use default values for that drink.**

#### Menu Categories and Default Options:

- **Hot-Only Drinks (Temperature: 핫 Only)**
  - 허브티 (Herbal Tea)  
    - Default: 미디움, 핫
    - Available Add-ons: None
  - 에스프레소 (Espresso)
    - Default: 미디움, 핫
    - Available Add-ons: [샷 추가]

- **Iced-Only Drinks (Temperature: 아이스 Only)**
  - 토마토주스 (Tomato Juice), 키위주스 (Kiwi Juice), 망고스무디 (Mango Smoothie), 딸기스무디 (Strawberry Smoothie), 레몬에이드 (Lemonade), 복숭아아이스티 (Peach Iced Tea)
    - Default: 미디움, 아이스
    - Available Add-ons: None
  - 아포카토 (Affogato)
    - Default: 미디움, 아이스
    - Available Add-ons: [샷 추가]
  - 쿠키앤크림 (Cookies and Cream)
    - Default: 미디움, 아이스
    - Available Add-ons: [휘핑크림]

- **Hot or Iced Drinks (Temperatures: 핫, 아이스)**
  - 카페라떼 (Cafe Latte), 바닐라라떼 (Vanilla Latte), 초콜릿라떼 (Chocolate Latte), 카푸치노 (Cappuccino), 아메리카노 (Americano), 카라멜마끼아또 (Caramel Macchiato), 카페모카 (Cafe Mocha), 말차라떼 (Matcha Latte)
    - Default: 미디움, 핫
    - Available Add-ons:
      - **카페라떼, 아메리카노**: [샷 추가]
      - **카푸치노**: [샷 추가, 휘핑크림]
      - **카라멜마끼아또**: [샷 추가, 카라멜시럽, 휘핑크림]
      - **바닐라라떼**: [샷 추가, 바닐라시럽, 휘핑크림]
      - **말차라떼, 초콜릿라떼, 카페모카**: [휘핑크림]

#### Important Rules for Processing Orders:

1. **Add-ons are Never Included by Default**: Only include add-ons if the customer **explicitly requests** them. If the customer does not mention an add-on, do not include it—even if the drink has available add-on options. **Add-ons must be completely omitted from the order unless specifically requested**.
2. **Default Drink Name for Generic Requests**: If the customer requests a general type of drink like "라떼" without specifying the type, default to 카페라떼.
3. **Combine Identical Items**: If the customer requests additional quantities of an item already in the order with the same details, combine them into a single entry with an updated quantity.

#### Extra Options
- 샷 (Shots): Add an extra shot to the drink.
- 카라멜시럽 (Caramel Syrup): Add caramel syrup for extra sweetness.
- 바닐라시럽 (Vanilla Syrup): Add vanilla syrup for flavor.
- 휘핑크림 (Whipped Cream): Add whipped cream as a topping.

### Two-Step Approach to Generating Responses:

1. **Step 1**: Analyze the customer input to determine the appropriate action type(s). 
   - **Identify Each Request Separately**: If the input includes multiple requests (e.g., add and update, or multiple add requests), treat each one as a separate action to be processed.
   - For each request, determine if it corresponds to one of the following actions:
     - **add_item**: If the customer is adding a new item to the order.
     - **update_item**: If the customer wants to change details of an existing item.
     - **remove_item**: If the customer is requesting to delete an item.
     - **cancel_order**: If the customer wants to cancel the entire order.
   - **Clarify Ambiguity**: If any request is unclear, add a response asking the customer for clarification.

2. **Step 2**: Construct each response in the correct format based on the identified action type. Each action should be formatted independently, and only include the necessary details for that action according to the guidelines below.

---

### Action Instructions and Formats:

1. **Add Item (`add_item`)**
   - **Usage**: Use `add_item` to add a new item to the order. **Include all fields except add-ons, unless add-ons are explicitly requested by the customer**. If the customer does not specify `size`, `temperature`, or `quantity`, use the default values for that drink.
   - **Important**: Do **not** include any add-ons by default, even if available for the drink, unless the customer specifically requests them.
   - **Format**: 
     ```
     add_item [drink] [size] [temperature] [quantity] | [add-ons]
     ```
   - **Examples**:
     - Full item with add-ons: If the customer requests “아이스 아메리카노 미디움 사이즈, 샷 추가 1개”:
       ```
       add_item 아메리카노 미디움 아이스 1 | (샷:1)
       ```
     - Item without add-ons: If the customer requests “핫 카푸치노 라지 사이즈 1잔”:
       ```
       add_item 카푸치노 라지 핫 1
       ```
     - **No add-ons included by default**: If the customer requests “카푸치노 두 잔 핫으로 주세요” (without mentioning add-ons), do not include add-ons in the response:
       ```
       add_item 카푸치노 미디움 핫 2
       ```
     - Using default values: If the customer requests “아메리카노” (without specifying size, temperature, or quantity):
       ```
       add_item 아메리카노 미디움 핫 1
       ```

2. **Update Item (`update_item`)**
   - **Usage**: Use `update_item` to modify specific details of an existing item in the order. **Only include fields specified by the customer**; do not add placeholders for missing fields. Use `|` separators only between fields with values.
   - **Format**:
     ```
     update_item indexes: [indexes] | [optional drink] | [size, temperature, quantity] | [add-ons]
     ```
   - **Examples**:
     - Updating only quantity: If the customer requests “인덱스 0의 아이템 수량을 2로 변경”:
       ```
       update_item indexes: 0 | | 2
       ```
     - Updating drink and quantity: If the customer requests “인덱스 0의 아이템을 아메리카노로 변경하고 수량을 3으로 설정”:
       ```
       update_item indexes: 0 | 아메리카노 | 3
       ```
     - Updating size, temperature, and quantity: If the customer requests “인덱스 1의 아이템을 라지, 아이스, 수량 2로 변경”:
       ```
       update_item indexes: 1 | | 라지, 아이스, 2
       ```
     - Updating all fields: If the customer requests “인덱스 0의 아이템을 바닐라라떼, 라지, 아이스, 수량 2, 바닐라시럽 및 휘핑크림 추가로 변경”:
       ```
       update_item indexes: 0 | 바닐라라떼 | 라지, 아이스, 2 | (바닐라시럽:1, 휘핑크림:1)
       ```

3. **Remove Item (`remove_item`)**
   - **Usage**: Use `remove_item` to delete one or more items from the order. Only include the `indexes` field with the item indexes to delete.
   - **Format**:
     ```
     remove_item indexes: [indexes]
     ```
   - **Examples**:
     - Removing a single item: If the customer requests “인덱스 0의 아이템을 삭제”:
       ```
       remove_item indexes: 0
       ```
     - Removing multiple items: If the customer requests “인덱스 1과 2의 아이템을 삭제”:
       ```
       remove_item indexes: 1, 2
       ```

4. **Cancel Order (`cancel_order`)**
   - **Usage**: Use `cancel_order` to clear the entire order. No additional fields are needed.
   - **Format**:
     ```
     cancel_order
     ```
   - **Example**:
     - Cancelling the entire order: If the customer requests “전체 주문을 취소”:
       ```
       cancel_order
       ```

---

### Important Notes
1. **Add-ons are Never Included by Default**: Only include add-ons if the customer explicitly requests them. Do not include any add-ons by default.
2. **Default Drink Name for Generic Requests**: For a general drink request like "라떼," add 카페라떼 to the order.
3. **Combine Identical Items**: For `add_item`, if the customer orders the same drink with identical details more than once, combine them into a single entry with the updated quantity.
4. **Multiple Requests**: If the input contains multiple valid requests, generate a separate action for each request.
5. **Update Item**: Only include fields specified by the customer. Do not add placeholders for missing fields. Use `|` separators only for fields with values.
6. **Remove Item**: Only include the `indexes` field listing the items to delete.
7. **Cancel Order**: Respond with only `cancel_order` and no additional details.

**Common Errors to Avoid**:
- Do not add placeholders for missing fields in `update_item`.
- Ensure fields are in the correct order in each format.
- **Never assume add-ons as default**; add-ons must be requested explicitly by the customer to be included.
- For multiple requests, do not combine actions unless they are identical. Each action should be independently formatted.
- Do not include any additional commentary or explanations.

### Instruction:
**Step 1**: Identify the correct action types based on the customer input. If there are multiple requests, handle each as a separate action. Then proceed to construct the responses for each action according to the correct format as described above.

When Current Order Details:
None
Previous customer inputs:
None

Generate the correct response in the specified format for the input below.

### Input:
카푸치노 두 잔 핫으로 주세요. 그리고 아메리카노 한 잔 아이스로 주세요.

### Expected Response:


In [ ]:
"""
1. Add Item (add_item)
Instruction: Use add_item to add a new item to the order. All fields must be included: drink, size, temperature, quantity, and any add-ons. Even if the customer doesn’t specify a field, ensure each field has a value in the response.

Format:

plaintext
Copy code
add_item [drink] [size] [temperature] [quantity] | [add-ons]
Examples:

Full item with add-ons: If the customer requests “아이스 아메리카노 미디움 사이즈, 샷 추가 1개”:

plaintext
Copy code
add_item 아메리카노 미디움 아이스 1 | (샷:1)
Item without add-ons: If the customer requests “핫 카푸치노 라지 사이즈 1잔”:

plaintext
Copy code
add_item 카푸치노 라지 핫 1
Explanation: In add_item, include all details about the drink: drink name, size, temperature, quantity, and add-ons (if any). Even if there are no add-ons, all other fields must be included.

2. Update Item (update_item)
Instruction: Use update_item to modify specific details of an existing item in the order. Only include fields specified by the customer. Do not add placeholders for missing fields. Use | separators only for fields with values.

Format:

plaintext
Copy code
update_item indexes: [indexes] | [optional drink] | [size, temperature, quantity] | [add-ons]
Examples:

Updating only quantity: If the customer requests “인덱스 0의 아이템 수량을 2로 변경”:

plaintext
Copy code
update_item indexes: 0 | | 2
Updating drink and quantity: If the customer requests “인덱스 0의 아이템을 아메리카노로 변경하고 수량을 3으로 설정”:

plaintext
Copy code
update_item indexes: 0 | 아메리카노 | 3
Updating size, temperature, and quantity: If the customer requests “인덱스 1의 아이템을 라지, 아이스, 수량 2로 변경”:

plaintext
Copy code
update_item indexes: 1 | | 라지, 아이스, 2
Updating all fields: If the customer requests “인덱스 0의 아이템을 바닐라라떼, 라지, 아이스, 수량 2, 바닐라시럽 및 휘핑크림 추가로 변경”:

plaintext
Copy code
update_item indexes: 0 | 바닐라라떼 | 라지, 아이스, 2 | (바닐라시럽:1, 휘핑크림:1)
Explanation: In update_item, only include fields that the customer wants to change. Do not add unnecessary placeholders (like | |). Use the | separator only between fields with values.

3. Remove Item (remove_item)
Instruction: Use remove_item to delete one or more items from the order. Include only the indexes field with the item indexes to delete.

Format:

plaintext
Copy code
remove_item indexes: [indexes]
Examples:

Removing a single item: If the customer requests “인덱스 0의 아이템을 삭제”:

plaintext
Copy code
remove_item indexes: 0
Removing multiple items: If the customer requests “인덱스 1과 2의 아이템을 삭제”:

plaintext
Copy code
remove_item indexes: 1, 2
Explanation: For remove_item, include only the indexes field listing all indexes of items to be removed. No other fields are required.

4. Cancel Order (cancel_order)
Instruction: Use cancel_order to clear the entire order. No additional fields are needed.

Format:

plaintext
Copy code
cancel_order
Example:

Cancelling the entire order: If the customer requests “전체 주문을 취소”:

plaintext
Copy code
cancel_order
Explanation: In cancel_order, respond with only the command cancel_order. No additional details or fields are needed.

Important Notes
Add Item: Always include all fields. For add_item, each field (drink, size, temperature, quantity, add-ons) must be present, even if some fields are not specified by the customer.
Update Item: Only include fields specified by the customer. If a field is not being updated, do not add it as a placeholder. Use | separators only for fields with values.
Remove Item: Only include the indexes field listing the items to delete.
Cancel Order: Respond with only cancel_order and no additional details.
Common Errors to Avoid:

Do not add placeholders for missing fields in update_item.
Ensure fields are in the correct order in each format.
Do not include any additional commentary or explanations.
By following these instructions, you will provide consistent and accurate responses for each order action in the coffee kiosk.
"""